# Batch Processing - Generate Cases

This notebook demonstrates creating cases in batch and running them in parallel.

## Create Cases in Batch

The approach to create cases in batch following this procedure:

- Load the base case from file
- For each desired case output:
  * Alter parameters to the desired value
  * Save each system to a new case file

In [1]:
import andes
import numpy as np
from andes.utils.paths import get_case

andes.config_logger(stream_level=30)  # brief logging

In [2]:
# create directory for output cases
!rm -rf batch_cases
!mkdir -p batch_cases

In [3]:
kundur = get_case('kundur/kundur_full.xlsx')
ss = andes.load(kundur)

Generating code for 1 models on 8 processes.


We demonstrate running the Kundur's system under different loading conditions.

Cases are created by modifying the `p0` of `PQ` with `idx == PQ_0`.

As always, input parameters can be inspected by accessing `Model.as_df(vin=True)`. 

In [4]:
p0_base = ss.PQ.get('p0', "PQ_0")

Create 3 cases so that the load increases from `p0_base` to `1.2 * p0_base`.

In [5]:
N_CASES = 3  # Note: increase `N_CASES` as necessary

p0_values = np.linspace(p0_base, 1.2 * p0_base, N_CASES)

In [6]:
for value in p0_values:
    ss.PQ.alter('p0', 'PQ_0', value)
    file_name = f'batch_cases/kundur_p_{value:.2f}.xlsx'
    
    andes.io.dump(ss, 'xlsx', file_name, overwrite=True)

## Parallel Simulation

Parallel simulation is easy with the command line tool.

Change directory to `batch_cases`:

In [7]:
import os

# change the Python working directory
os.chdir('batch_cases')

In [8]:
!ls -la

total 96
drwxr-xr-x@  5 hcui7  staff    160 Jan  4 21:38 .
drwxr-xr-x  16 hcui7  staff    512 Jan  4 21:38 ..
-rw-r--r--@  1 hcui7  staff  14855 Jan  4 21:38 kundur_p_11.59.xlsx
-rw-r--r--@  1 hcui7  staff  14856 Jan  4 21:38 kundur_p_12.75.xlsx
-rw-r--r--@  1 hcui7  staff  14854 Jan  4 21:38 kundur_p_13.91.xlsx


### Running from Command line

In [9]:
!andes run *.xlsx -r tds


    _           _         | Version 1.9.3.post20+gd85520b62.d20260104
   /_\  _ _  __| |___ ___ | Python 3.11.14 on Darwin, 01/04/2026 09:38:47 PM
  / _ \| ' \/ _` / -_|_-< | 
 /_/ \_\_||_\__,_\___/__/ | This program comes with ABSOLUTELY NO WARRANTY.

Working directory: "/Users/hcui7/repos/andes/examples/batch_cases"


> Loaded generated Python code in "/Users/hcui7/.andes/pycode".
-> Processing 3 jobs on 8 CPUs.
Process 0 for "kundur_p_11.59.xlsx" started.
Process 1 for "kundur_p_12.75.xlsx" started.
Process 2 for "kundur_p_13.91.xlsx" started.


Generating code for 1 models on 8 processes.
Generating code for 1 models on 8 processes.
Generating code for 1 models on 8 processes.


  0%|                                                    | 0/100 [00:00<?, ?%/s]

<Toggle 1>: Line.Line_8 status changed to 0 at t=2.0 sec.
 10%|####                                    | 10.0/100 [00:00<00:00, 161.45%/s]

...

 74%|#############################6          | 74.0/100 [00:00<00:00, 104.18%/s]

 88%|###################################2    | 88.0/100 [00:00<00:00, 117.82%/s]

 88%|###################################2    | 88.0/100 [00:00<00:00, 101.96%/s]

100%|#######################################| 100.0/100 [00:00<00:00, 105.08%/s]
100%|#######################################| 100.0/100 [00:00<00:00, 106.03%/s]
100%|#######################################| 100.0/100 [00:00<00:00, 104.29%/s]
/Users/hcui7/miniforge3/envs/aa/lib/python3.11/multiprocessing/resource_tracker.py:254: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '


/Users/hcui7/miniforge3/envs/aa/lib/python3.11/multiprocessing/resource_tracker.py:254: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '
Log saved to "/var/folders/45/yt95k1qd4cj0wt64977wkb8w0000gr/T/andes-ne7ttrnc/andes.log".
-> Multiprocessing finished in 2.0138 seconds.
/Users/hcui7/miniforge3/envs/aa/lib/python3.11/multiprocessing/resource_tracker.py:254: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '


### Number of CPUs

In some cases, you don't want the simulatino to use up all resources. 

ANDES allows to control the number of processes to run in parallel through `--ncpu NCPU`, where `NCPU` is the maximum number of processes (equivalent to the number of CPU cores) allowed. 

In [10]:
!andes run *.xlsx -r tds --ncpu 4


    _           _         | Version 1.9.3.post20+gd85520b62.d20260104
   /_\  _ _  __| |___ ___ | Python 3.11.14 on Darwin, 01/04/2026 09:38:50 PM
  / _ \| ' \/ _` / -_|_-< | 
 /_/ \_\_||_\__,_\___/__/ | This program comes with ABSOLUTELY NO WARRANTY.

Working directory: "/Users/hcui7/repos/andes/examples/batch_cases"
> Loaded generated Python code in "/Users/hcui7/.andes/pycode".
-> Processing 3 jobs on 4 CPUs.
Process 0 for "kundur_p_11.59.xlsx" started.
Process 1 for "kundur_p_12.75.xlsx" started.
Process 2 for "kundur_p_13.91.xlsx" started.


Generating code for 1 models on 8 processes.
Generating code for 1 models on 8 processes.
Generating code for 1 models on 8 processes.


  0%|                                                    | 0/100 [00:00<?, ?%/s]

<Toggle 1>: Line.Line_8 status changed to 0 at t=2.0 sec.
<Toggle 1>: Line.Line_8 status changed to 0 at t=2.0 sec.
<Toggle 1>: Line.Line_8 status changed to 0 at t=2.0 sec.
 10%|####                                    | 10.0/100 [00:00<00:00, 148.39%/s]

 14%|#####6                                  | 14.0/100 [00:00<00:00, 128.79%/s]

...

 54%|#####################6                  | 54.0/100 [00:00<00:00, 126.98%/s]

 67%|##########################8             | 67.0/100 [00:00<00:00, 117.75%/s]

 80%|################################        | 80.0/100 [00:00<00:00, 120.22%/s]

100%|#######################################| 100.0/100 [00:00<00:00, 128.68%/s]
100%|#######################################| 100.0/100 [00:00<00:00, 128.27%/s]


100%|#######################################| 100.0/100 [00:00<00:00, 125.68%/s]
/Users/hcui7/miniforge3/envs/aa/lib/python3.11/multiprocessing/resource_tracker.py:254: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '
/Users/hcui7/miniforge3/envs/aa/lib/python3.11/multiprocessing/resource_tracker.py:254: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '
Log saved to "/var/folders/45/yt95k1qd4cj0wt64977wkb8w0000gr/T/andes-gmrcmyi6/andes.log".
-> Multiprocessing finished in 1.5771 seconds.
/Users/hcui7/miniforge3/envs/aa/lib/python3.11/multiprocessing/resource_tracker.py:254: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '


### Running with APIs

Setting `pool = True` allows returning all system instances in a list. 

This comes with a penalty in computation time but can be helpful if you want to extract data directly.

In [11]:
systems = andes.run('*.xlsx', routine='tds', pool=True, verbose=10)

Cases are processed in the following order:
"kundur_p_11.59.xlsx"
"kundur_p_12.75.xlsx"
"kundur_p_13.91.xlsx"


Converged in 1 steps for t=0.000000s, h=0.033333s, max inc=0 

Converged in 1 steps for t=0.000000s, h=0.033333s, max inc=0 

Converged in 1 steps for t=0.000000s, h=0.033333s, max inc=0 

Converged in 1 steps for t=0.033333s, h=0.033333s, max inc=0 

...

Converged in 3 steps for t=19.933433s, h=0.033333s, max inc=7.823e-05 

Converged in 3 steps for t=20.000000s, h=0.033233s, max inc=1.238e-05 

Converged in 3 steps for t=19.966767s, h=0.033333s, max inc=7.524e-05 

Converged in 3 steps for t=20.000000s, h=0.033233s, max inc=7.103e-05 

Log saved to "/var/folders/45/yt95k1qd4cj0wt64977wkb8w0000gr/T/andes-70286q45/andes.log".
-> Multiprocessing finished in 11.5948 seconds.


In [12]:
systems[0]

In [13]:
systems

### Example plots

Plotting or data analyses can be carried out as usual.

In [14]:
ss = systems[0]

In [15]:
systems[0].TDS.plotter.plot(ss.GENROU.omega, latex=False)

/Users/hcui7/repos/andes/andes/plot.py:822: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


(<Figure size 640x480 with 1 Axes>, <Axes: xlabel='Time [s]'>)

In [16]:
systems[2].TDS.plotter.plot(ss.GENROU.omega, latex=False)

/Users/hcui7/repos/andes/andes/plot.py:822: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


(<Figure size 640x480 with 1 Axes>, <Axes: xlabel='Time [s]'>)

In [17]:
!andes misc -C
!rm -rf batch_cases

No output file found in the working directory.
